In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab05.ipynb")

# Lab 5: Gibbs Sampling and Linear Models
Welcome to the Data 102 Lab 5. In this lab, we are going to wrap up our discussion of approximate inference methods with Gibbs Sampling, and give you a chance to build GLMs from a Bayesian and Frequentist perspective.

#### The code and responses you need to write are are represented by `...`. There is additional documentation for each part as you go along.

##### Please read carefully the introduction and the instructions to each problem.

## Collaboration Policy
Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the cell below.

**Collaborators:**

## Submission
See the [Gradescope Submission Guidelines](https://edstem.org/us/courses/42657/discussion/3350112) for details on how to submit your lab.

Again, since this lab involves sampling, **tests may take awhile to run. Please submit as early as possible, as last minute submissions may overwhelm datahub, preventing yourself and others from submitting on-time.**

**For full credit, this assignment should be completed and submitted before Wednesday, October 4th, 2023 at 11:59 PM PST.**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from ipywidgets import interact, interactive
import itertools
import hashlib
from scipy.stats import poisson, norm, gamma, uniform, multivariate_normal
#!pip install pymc3
import statsmodels.api as sm

from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
  
sns.set(style="dark")
plt.style.use("ggplot")

from pymc import *
import pymc as pm
import bambi as bmb

import arviz as az

# Part I: Gibbs Sampling

As in Lab 4 Question 1, you are given a two-dimensional unnormalized density function $q(x,y)$ represented by `target_density` below. At the end of that question, you were asked to reflect on the inefficiency of rejection sampling in high-dimensional settings. Here, we consider an alternative sampling method called **Gibb's Sampling**.

In this question, you'll have the chance to implement Gibb's Sampling using a 1-D rejection sampling subroutine and compare its performance to that of rejection sampling.

*Throughout this question we will assume that our computers have access only to normal and uniform random variables.*

### Importing Setup from Lab 4

Here, we start by importing the setup to Lab 4 Question 1.  

In [ ]:
# This is the target unnormalized density from which we would like to sample
# Run this to define the function
# No TODOs here
@np.vectorize # <- decorator, makes function run faster
def target_density(x, y):
    mean1 = [1, 1.7]
    mean2 = [2, 1.3]
    mean3 = [1.5, 1.5]
    mean4 = [2, 2.1]
    mean5 = [1, 1.2]
    cov1=0.2*np.array([[0.2, -0.05], [-0.05, 0.1]])
    cov2 = 0.3*np.array([[0.1, 0.07], [0.07, 0.2]])
    cov3= np.array([[0.1, 0], [0, 0.1]])
    cov4 = 0.1*np.array([[0.3, 0.04], [0.04, 0.2]])
    cov5 = 0.1*np.array([[0.4, -0.04], [-0.04, 0.2]])
    return(multivariate_normal.pdf([x, y], mean=mean1, cov=cov1) + 
           multivariate_normal.pdf([x, y], mean=mean2, cov=cov2) +
           2*multivariate_normal.pdf([x, y], mean=mean3, cov=cov3) +
           0.5*multivariate_normal.pdf([x, y], mean=mean4, cov=cov4)+
           0.5*multivariate_normal.pdf([x, y], mean=mean5, cov=cov5))

#### Let's visualize this density. 

Run the cell below to see a 3D plot of the function along with a contour plot.

In [ ]:
# No TODOs here, just run the cell to make plots
# Create a meshgrid of coordinates
coords = np.arange(0.5, 2.5, 0.02)
X, Y = np.meshgrid(coords, coords)

# Compute the value of the target density at all pairs of (x,y) values
Z = target_density(X,Y)

In [ ]:
# Display the 3D plot of the target density
fig = plt.figure(figsize=(15,6))

ax0 = fig.add_subplot(121, projection='3d')
ax1 = fig.add_subplot(122)

surf = ax0.plot_surface(X,Y,Z, cmap=cm.plasma, linewidth=0, antialiased=False,alpha = 0.9,)

# Customize the z axis.
ax0.set_zlim(0, 7)
ax0.set_xlabel("X")
ax0.set_ylabel("Y")
ax0.set_zlabel("Z")
ax0.set_title("3D plot of the target density")

# Rotate the axes: you can change these numbers in order to see the distribution from other angles
ax0.view_init(50, 25)

# Plot the contour plot of the density
cont = ax1.contour(X,Y,Z, levels = 20, cmap=cm.plasma, linewidths=1)
ax1.set_xlabel("X")
ax1.set_ylabel("Y")
ax1.set_title("Contour plot of the target density")

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5, ax=ax1)
plt.tight_layout()
plt.show()

Take a moment to examine the plots. Make sure you can see correspondances between each peak in the 3D plot on the left; and the "high-altitude" regions in the countour plot on the right.

Next we will plot 1-dimensional projections of the target densities onto the $X$ and $Y$ axis. These correspond to conditional target distributions of the form $q(x, y=y')$ and $q(x=x', y)$.

In [ ]:
# Do not modify
# Run the cell below to define the plotting functions

COORDINATES = np.arange(0, 3, 0.02)
def plot_x_cond(y_val):
    fig, axs = plt.subplots(1, 2)
    fig.set_figheight(5)
    fig.set_figwidth(12)
    axs[0].contour(X,Y,Z, levels = 20, cmap=cm.plasma, alpha = 0.8, linewidths=0.8)
    axs[0].axhline(y_val,  ls="--", color = 'olive', lw = 2)
    axs[0].set_xlabel("X")
    axs[0].set_ylabel("Y")
    axs[0].set_title("Contour plot of the target density")
    
    axs[1].plot(COORDINATES, target_density(COORDINATES, y_val), color = 'olive')
    axs[1].set_ylim(0,10)
    axs[1].set_xlim(0,3)
    axs[1].set_xlabel("X")
    axs[1].set_title("Conditional target density: q(x | y={:.1f})".format(y_val))
    plt.show()
    
def plot_y_cond(x_val):
    fig, axs = plt.subplots(1, 2)
    fig.set_figheight(5)
    fig.set_figwidth(12)
    axs[0].contour(X,Y,Z, levels = 20, cmap=cm.plasma, alpha = 0.8, linewidths=0.8)
    axs[0].axvline(x_val,  ls="--", color = 'olive', lw = 2)
    axs[0].set_xlabel("X")
    axs[0].set_ylabel("Y")
    axs[0].set_title("Contour plot of the target density")
    
    axs[1].plot(COORDINATES, target_density(x_val, COORDINATES), color = 'olive')
    axs[1].set_ylim(0,10)
    axs[1].set_xlim(0,3)
    axs[1].set_xlabel("Y")
    axs[1].set_title("Conditional target density: q(y | x={:.1f})".format(x_val))
    plt.show()

In [ ]:
# Display interactive plot
interactive_plot = interactive(plot_x_cond, y_val=(0, 3, 0.1), add_proposal=False)
interactive_plot

## Importing Rejection Sampling Helper Functions

From Lab 4 Question 1, we also bring back the helper functions that you defined in the question. These will be helpful later when we implement our Gibb's sampler!

In [ ]:
def sample_1D_proposed_distribution(N):
    """ 
    Produces N samples from the Uniform(0,3) proposal distribution
    
    Inputs:
        N : int, desired number of samples
        
    Outputs:
        proposed_samples : an 1d-array of size N which contains N independent samples from the proposal
    """
    
    proposed_samples = uniform.rvs(0,3, N) 
    return(proposed_samples)

@np.vectorize
def compute_ratio_1D(proposed_sample, c):
    """
    Computes the ratio between the scaled target density and proposal density evaluated at the 
    proposed sample point
    
    Inputs:
        proposed_sample : float, proposed sample
        c : float, constant scaling factor that ensures that the proposal density is above the target density
        
    Outputs:
        ratio : float
    """

    ratio = target_1D_density(proposed_sample)*c / (1/3) 
    assert(ratio <= 1)
    return(ratio)

@np.vectorize
def accept_proposal(ratio):
    """ 
    Accepts or rejects a proposal with probability equal to ratio
    
    Inputs: 
        ratio: float, probability of acceptance
    
    Outputs:
        accept: True/False, if True, accept the proposal
    """
    gamma = uniform.rvs(0, 1)
    accept = gamma <= ratio
    return(accept)

## Question 1: Gibbs Sampling

In this question, we will build a Gibbs sampler and apply it to the same density. First, let's go over the basics of Gibbs Sampling.

Assume we want to sample from an unnormalized target density $q(x, y)$. 

#### Gibbs Sampling proceeds as follows:

- Start at an initial point $(x_0, y_0)$
- For `i` in `number of iterations`:
    - Condition on $y=y_{i-1}$: Sample $x_i \sim q(x| y=y_{i-1})$ 
        - Add $(x_i, y_{i-1})$ to the list of samples
    - Condition on $x=x_{i}$: : Sample $y_i \sim q(y| x=x_{i})$ 
        - Add $(x_i, y_{i})$ to the list of samples
    
In many problems we can sample the univariate distributions directly. In this case we don't know how to sample them directly, but we can use the 1-D rejection sampler that we computed in Lab 4 Question 1 (functions imported above).

In the cell below, we wrote helper functions that sample from the conditionals above. They are essentially the same function you wrote in Lab 4, just slightly modified such that we perform rejection sampling until we get one valid sample.

In [ ]:
# No TODOs here:
# Just look at these helper functions and make sure you understand the syntax

def sample_x_cond(fixed_y_val):
    """ 
    Produces one sample from x_i ~ q(x | y=fixed_y_val)
    
    Inputs:
        fixed_y_val : float, current value of y, on which we condition
    
    Outputs:
        x_sample: float, one sample from x_i ~ q(x, y=fixed_y_val)
        num_samples : int, number of tries until we accepted a sample
        
    """
    def conditional_density(x):
        return(target_density(x, fixed_y_val))
    x_sample = None
    num_samples = 0
    c = 0.33/(0.2+max(conditional_density(np.arange(0.5, 2.5, 0.05)))) # <- we are cheating a bit here by 
                                                                       # looking for a tight c value
    while x_sample is None:
        proposed_sample = sample_1D_proposed_distribution(1)
        num_samples += 1
        ratio = conditional_density(proposed_sample)*3*c
        assert(ratio <= 1)
        accept = accept_proposal(ratio)
        if accept:
            x_sample = proposed_sample[0]
    return(x_sample, num_samples)


def sample_y_cond(fixed_x_val):
    """ 
    Produces one sample from y_i ~ q(y | x=fixed_x_val)
    
    Inputs:
        fixed_x_val : float, current value of y, on which we condition
    
    Outputs:
        y_sample: float, one sample from y_i ~ q(y | x=fixed_x_val)
        num_samples : int, number of tries until we accepted a sample
        
    """
    def conditional_density(y):
        return(target_density(fixed_x_val, y))
    y_sample = None
    num_samples = 0
    c = 0.33/(0.2+max(conditional_density(np.arange(0.5, 2.5, 0.05))))
    while y_sample is None:
        proposed_sample = sample_1D_proposed_distribution(1)
        num_samples += 1
        ratio = conditional_density(proposed_sample)*3*c
        assert(ratio <= 1)
        accept = accept_proposal(ratio)
        if accept:
            y_sample = proposed_sample[0]
    return(y_sample, num_samples)

### 1a) Build a Gibbs sampler using the helper functions above
**Note**: Don't forget that at each iteration the Gibbs sampler adds two samples to the list of samples: $(x_i, y_{i-1})$ and $(x_i, y_{i})$

**Hint 1**: If you cannot pass the test, try making a simple test case with `N=2` and adding some print statements to your code.

**Hint 2**: In our implementation of gibbs sampling, `num_samples` is not trivially equal to `N`, since sampling $x_i \sim f(x| y=y_{i-1})$ and $y_i \sim f(y| x=x_{i})$ rely on rejection sampling.

In [ ]:
def get_2D_Gibbs_samples(N, x_0, y_0):
    """
    Produces N samples from the target density using Gibbs Sampling
    
    Inputs: 
        N : desired number of samples
        x_0, y_0 : floats, the coordinates of the starting point
        
    Outputs:
        gibbs_samples : array of dimension (N, 2) where each row is a sample from the target distribution
                        of the form (x_i, y_i)
        num_samples : total number of samples required
    """
    gibbs_samples = [] # Each entry corresponds to a (x_i, y_i)
    num_samples = 0 # Add the number of samples to this variable, note this is not equal to N since rejection sampling
                    # does not accept every sample
    x_curr = x_0 # Current value of x, initialized to x_0
    y_curr = y_0 # Current value of y, initialized to y_0
    
    for i in range(N//2): # The range is N//2 since we are generating two gibbs samples in one iteration
        ...
        
    return(gibbs_samples, num_samples)

In [ ]:
grader.check("q1a")

### 1b) Path traced by the Gibbs sampler
Run the code below to overlay the path traced by the Gibbs Sampler:

In [ ]:
# Do not modify
# Just run this once you've passed the validation tests above
N = 20
target_samples, total_samples = get_2D_Gibbs_samples(N, 1, 1)
target_samples = np.array(target_samples)

fig = plt.figure(figsize=(12,8))

# Plot the contour plot of the density
cont = plt.contour(X,Y,Z, levels = 20, cmap=cm.plasma, linewidths=1, alpha = 0.8)
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Path of the Gibbs Sampler")

# Add sample points obtained via Gibbs sampling
plt.scatter(target_samples[:,0], target_samples[:,1], c='b', alpha = 1, s=50, label = 'Samples')
for i in range(N):
    plt.annotate(i, (target_samples[i,0], target_samples[i,1]), fontsize = 20)
plt.plot(target_samples[:,0], target_samples[:,1], c='r', alpha = 1, label = 'Path of the Gibbs Sampler')

plt.legend()
plt.tight_layout()
plt.show()

<!-- BEGIN QUESTION -->

#### Inspect the scatter plot above. Trace the Gibbs sampler path from the initial point (labeled 0) to the final point. What do you notice about the orientation of the paths between each point? Why are they oriented in this way?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### 1c) 'Efficiency' of Gibbs Sampling
Let's compute 1000 Gibbs samples and compute how many times the rejection sampling subroutine accepted the proposed sample (running this might take a little while):

In [ ]:
# No need to modify this
# just run it and comment in the section below
N = 1000
target_samples, total_samples = get_2D_Gibbs_samples(N, 1, 1)
acceptance_rate = N/total_samples*100
print("The acceptance rate for Gibbs Sampling is {:.1f}%".format(acceptance_rate))

<!-- BEGIN QUESTION -->

#### How does Gibbs Sampling compare to vanilla Rejection Sampling from Lab 4? Is this approach more efficient or less efficient? What is the source of this increase/decrease in efficiency?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

# Part II: GLM

Now, we will pivot to discussing frequentist and Bayesian approaches to generalized linear models.
## Question 2: Atlantic Hurricane Season

With 30 named storms, the 2020 Atlantic hurricane season was the most active on record. Climate scientists argue that the culprit is human induced global warming. There is a an evergrowing body of research linking increased average temperatures and rising sea levels to more frequent, more intense and more destructive storms. 

In this lab we will investigate the number of named storms recorded since 1880, and we will argue that there is a statistically significant relationship between rising Sea Surface Temperature (SST) and the frequency of named storms.

For this lab we extracted the number of tropical storms from the [HURDAT Database](https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2019-052520.txt). We also extracted data on Sea Surface Temperatures from the [National Center for Atmosferic Research](https://climatedataguide.ucar.edu/climate-data/global-surface-temperature-data-gistemp-nasa-goddard-institute-space-studies-giss). 

### Load the data

In [ ]:
# No need to modify: Just run the code to load the data
data_source = "hurricane_data.csv"
df = pd.read_csv(data_source)
df = df[["Year", "Num_Storms", "Temp_Anomaly"]]
df.tail()

### Model Specifications

The `Num_Storms` column contains the number of named storms recorded each year between 1880 and 2019. The `Temp_Anomaly` column contains the deviation in yearly SST from the mean of 1951-1980.

In this question, to show that there is a statistically significant relationship between rising Sea Surface Temperature (SST) and the frequency of named storms, we will model the number of named storms in Year $i$ using **Poisson Regression**:

$$\lambda_i = e^{q_0 + q_1 X_i}$$ 

$$C_i \sim \text{Poisson}(\lambda_i),$$

where $X_i$ is the SST deviation in Year $i$, and $C_i$ is the number of named storms in year $i$.

This isn't something that we can easily solve from scratch, so we have to use software packages. In this question, we'll explore the two approaches to GLMs that we covered in class: 

1. **(Q2b) Frequentist Regression** using [`statsmodels.api`](https://www.statsmodels.org/stable/glm.html) 
2. **(Q2c) Bayesian Regression** via sampling using [`PyMC`](https://www.pymc.io/welcome.html) and [`Bambi`](https://bambinos.github.io/bambi/)

<!-- BEGIN QUESTION -->

## 2a) Understanding Check

The model we described above is a GLM. What is "Linear" about this GLM model? What's the inverse link function? 


_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 2b) Frequentist Regression

Let's start by considering the problem from a frequentist lens. To do this, we'll use the `statsmodels.api`, which allows us to create a model in just a few lines of code.

After fitting our model, we can call the `.summary()` method, and get a breakdown of our model and some details on how well it fit our data.

In [ ]:
# Fit Poisson GLM model where Temp_Anomaly is a covariate (exogenous variable): No need to modify
freq_model = sm.GLM(df["Num_Storms"], exog = sm.add_constant(df["Temp_Anomaly"]), 
                  family=sm.families.Poisson())
freq_res = freq_model.fit()
print(freq_res.summary())

<!-- BEGIN QUESTION -->

### 2bi) Understanding the table

What variable does `Temp_Anomaly`'s `coef` in the table correspond to in our model? 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 2bii) Inspecting the results of fitting `freq_model`. 

Does the model suggest that increased SST relate to more storms? Is the influnce of SST on number of storms statistically significant?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 2c) Bayesian Regression via PyMC

Now that we've done Poisson regression the frequentist way with the `statsmodels` package, let's try implementing it the Bayesian way! In this lab we'll explore two ways of doing this:
1. Building the model from scratch in `PyMC`
2. Using `Bambi`, a wrapper on `PyMC` that simplifies model construction

To start, let's build our model from scratch in `PyMC`. Unlike the `statsmodels` package, `PyMC` requires us to specify our model piece by piece. That means that, as with any Bayesian parameter estimation task, we have to distill our problem setup into a likelihood and prior before we can proceed.

### 2ci) Unpacking $C_i \sim \text{Poisson}(\lambda_i)$

Recall the problem setup:

Our model involves the relationship between rising Sea Surface Temperature (SST) and the frequency of named storms, where:

$$\lambda_i = e^{q_0 + q_1 X_i}$$ 

$$C_i \sim \text{Poisson}(\lambda_i),$$

where $X_i$ is the SST deviation in Year $i$, and $C_i$ is the number of named storms in year $i$.

**In the cell below, Choose the option that best fills in the blank in the following statement:**

$C_i \sim \text{Poisson}(\lambda_i)$ represents our _____:

**A.** Prior

**B.** Likelihood

**C.** Posterior

Your answer should be a string, either `"A"`, `"B"`, or `"C"`.

In [ ]:
q2c_i = ...

In [ ]:
grader.check("q2c_i")

<!-- BEGIN QUESTION -->

### 2cii) Picking our prior(s)

Now, let's examine the parameter $\lambda_i = e^{q_0 + q_1 X_i}$. As discussed in lecture, $\lambda_i$ is comprised of a linear function of our observations ($q^TX$) and an inverse-link function ($e$). Given the setup of our problem, answer the following questions using 1 sentence each:
1. How many prior distributions do we need to define?
2. What parameters will we define these prior distributions for?
3. Since we don't have any prior information about these random variables, what distribution (i.e Normal, Beta, etc.) should we pick for their priors?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### 2civ) Defining our PyMC Model
Given your answers to `q2c_i` and `q2c_ii`, you're now ready to make your PyMC model! Fill out the code cell below to build your model and sample from the posterior.

**Note:** To pass the test, 
1. Do not remove the variables currently present in the model and/or add your own additional variables. Just fill in any ellipsis; the variables defined here are more than enough for you to solve the question!
2. Make sure the name parameter you pass to each `Distribution` object matches the variable name it's assigned to. Your answers should follow the following format: 

```
with pm.Model() as model:

    q0 = pm.Some_Distribution('q0', ...)
    q1 = pm.Some_Distribution('q1', ...)`
    ...
```

**Hint 1**: Remember that random variables can be added, subtracted, multiplied, etc. just like normal numbers!

**Hint 2**: Not all variables defined in a model context necessarily have to be defined using `pm.Some_Distribution(...)`. In particular, we do not need `pm.Deterministic` when defining `lam`, since we're not interested in its posterior samples. 

In [ ]:
with pm.Model() as model:
    q0 = ...
    q1 = ...

    lam = ...
    
    Y_obs = ...

    # DO NOT CHANGE THE SAMPLING ROUTINE
    trace = pm.sample(2000, chains = 4, random_seed = 0, return_inferencedata = True)

In [ ]:
grader.check("q2c_iv")

Now that we've ran PyMC, we can visualize the posterior distributions of $q_0$ and $q_1$ and find our estimates $\hat{q}_0$ and $\hat{q}_1$.

In [ ]:
az.plot_posterior(trace, round_to=3)
plt.show()

## 2d) Bayesian Regression via Bambi

Whew! `q2c` took *a lot* of code just to build a simple Poisson regression. In practice, building models from scratch like this is usually unnecessary unless we're looking for a custom solution. Instead, we can rely on packages like `Bambi` that *wrap* the functionality of `PyMC` with a simpler interface purely designed for GLMs.

Using [`Bambi` documentation](https://bambinos.github.io/bambi/) as a guide, fill out the code cell below to fit a Bayesian Poisson Regression model on our data. 

**Note 1**: To pass the autograder, make sure you pass your model the `random_seed = 0` when you fit it!

**Note 2**: Notice that the `Bambi` package doesn't need us to specify a prior: when we don't specify a prior for our model parameters, Bambi automatically supplies a weakly informative one based on your data by default. For this question, we are okay with this behavior: **to pass the test, do not specify a prior on your parameters!**

In [ ]:
my_model = ...
my_model_samples = ...

az.plot_posterior(my_model_samples, round_to=3)
plt.show()

In [ ]:
grader.check("q2d")

<!-- BEGIN QUESTION -->

### 2e) Understanding the plots
What the are x-axis and y-axis in each of the plots in 2d? Your answer should be in terms of the parameters of our model.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 2f) Comparison

Compare the results of `freq_model` in 2b with the plot in 2d. Are the estimates of Frequentist and Bayesian Regression close to each other? 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Congratulations! You have finished Lab 5! ##

Below, you will see two cells. Running the first cell will automatically generate a PDF of all questions that need to be manually graded, and running the second cell will automatically generate a zip with your autograded answers. **You are responsible for both the coding portion (the zip from Lab 5) and the written portion (the PDF of written responses from Lab 5) to their respective Gradescope portals.** The coding proportion should be submitted to the `Lab 5` assignment as a single zip file, and the written portion should be submitted to `Lab 5 PDF` assignment as a single pdf file. When submitting the written portion, please ensure you select pages appropriately.

If there are issues with automatically generating the PDF in the first cell, you can try downloading the notebook as a PDF by clicking on `File -> Save and Export Notebook As... -> PDF`. If that doesn't work either, you can manually take screenshots of your answers to the manually graded questions and submit those. Either way, **you are responsible for ensuring your submission follows our requirements, we will NOT be granting regrade requests for submissions that don't follow instructions.**

In [ ]:
import matplotlib.image as mpimg
from otter.export import export_notebook
from os import path
from IPython.display import display, HTML
export_notebook("lab05.ipynb", filtering=True, pagebreaks=True)
if(path.exists('lab05.pdf')):
    img = mpimg.imread('chinchilla.jpg')
    imgplot = plt.imshow(img)
    imgplot.axes.get_xaxis().set_visible(False)
    imgplot.axes.get_yaxis().set_visible(False)
    plt.show()
    display(HTML("Download your PDF <a href='lab05.pdf' download>here</a>."))
else:
    print("\n Pdf generation fails, please try the other methods described above")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)